In [ ]:
from datetime import datetime, timedelta, timezone
import os
import pandas as pd
import xarray as xr

from ONCToolbox import ONCToolbox, nan_onc_flags
from ONCToolbox.utils.suna import SUNAv2ArchiveFileParser as SAFP

In [ ]:
date_to = datetime.now(timezone.utc)
date_from = date_to - timedelta(days = 150)

In [ ]:
onct = ONCToolbox()
safp = SAFP()

In [ ]:
%%time
remote_files = onct.find_archive_file_urls(location_code = 'BACVP', device_category_code='NITRATESENSOR', date_from = date_from, date_to= date_to)
ds_list = []
for remote_file in remote_files:
    lines = safp.import_suna_from_url(remote_file)
    ds = safp.process_suna_lines(lines)
    ds_list.append(ds)
suna = xr.combine_by_coords(ds_list, combine_attrs='drop_conflicts')
suna.to_netcdf(f'data/bacvp_suna_{date_from.strftime("%Y%m%d")}-{date_to.strftime("%Y%m%d")}.nc')

In [ ]:
%%time
dtr = pd.date_range(start=date_from, end=date_to, freq='1D')

for dt in dtr:
    bdt = dt.replace(hour=0, minute=0, second=0, microsecond=0)
    edt = dt.replace(hour=23, minute=59, second=59, microsecond=999999)
    fname = f'data/bacvp_ctd_{bdt.strftime("%Y%m%d")}.nc'
    if os.path.isfile(fname):
        continue
    try:
        ctd = onct.get_fullres_data(location_code = 'BACVP', 
                                    device_category_code = 'CTD', 
                                   date_from = bdt, 
                                   date_to = edt, 
                                   out_as = 'xarray',
                                   add_metadata = True)
        ctd = nan_onc_flags(ctd)
        ctd.to_netcdf(fname)
        print(fname)
    except:
        continue